In [ ]:
## Do this only in Colab notebooks! Otherwise use pip install unsloth
#!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
#!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
#!pip install --no-deps unsloth

In [1]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
import pandas as pd
from pathlib import Path
import pickle
import json
import time
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
#model.save_pretrained("models/qwe2_5_3b_full")
#tokenizer.save_pretrained("models/qwe2_5_3b_full")

[]

In [ ]:
#from transformers import AutoTokenizer
#
## Указываем путь к сохраненной модели и токенизатору
#model_path = "models/qwe2_5_3b_full"
#
## Загружаем модель и токенизатор
#model = FastVisionModel.from_pretrained(model_path, load_in_4bit=True, use_gradient_checkpointing="unsloth")
#tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
#!wget  -c "https://docs.google.com/spreadsheets/d/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD/export?format=xlsx" -O "проверка ценников.xls"

--2025-05-06 06:37:28--  https://docs.google.com/spreadsheets/d/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 74.125.205.194, 2a00:1450:4010:c02::c2
Connecting to docs.google.com (docs.google.com)|74.125.205.194|:443... connected.
HTTP request sent, awaiting response... 

307 Temporary Redirect
Location: https://doc-0k-3k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/1a07496inq9bch4p8p9pj9da3c/1746513445000/113369956649761615778/*/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD?format=xlsx [following]
--2025-05-06 06:37:28--  https://doc-0k-3k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/1a07496inq9bch4p8p9pj9da3c/1746513445000/113369956649761615778/*/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD?format=xlsx
Resolving doc-0k-3k-sheets.googleusercontent.com (doc-0k-3k-sheets.googleusercontent.com)... 64.233.164.132, 2a00:1450:4010:c07::84
Connecting to doc-0k-3k-sheets.googleusercontent.com (doc-0k-3k-sheets.googleusercontent.com)|64.233.164.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘проверка ценников.xls’

проверка ценников.x     [ <=>                ]  53.79K  --.-KB/s    in 0.04s   

2025-05-06 06:37:28 (1.37 MB/s) - 

In [4]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [5]:
df = pd.read_excel("/data/проверка ценников.xlsx", header=1)

In [6]:
df.head()

,Номер картинки,Цена правильная,Цена нейросеть,Ошибка,Строка 1,Строка 2,Строка 3,Строка 4,Строка 5
0,1,54.99,54.99,0.0,Вода СВЯТОЙ ИСТОЧНИК,1.0л,природная питьевая негаз.ПЭТ,NaN,NaN
1,2,59.99,59.99,0.0,Вода Святой источник,"0,75л",негазированная спорт,NaN,NaN
2,3,51.99,51.99,0.0,Вода ШИШКИН ЛЕС,питьевая 1л,негазированная,NaN,NaN
3,4,24.99,24.99,0.0,Вода КРАСНАЯ ЦЕНА,"1,5л",негазированная,NaN,NaN
4,5,61.99,61.99,0.0,Вода СВЯТОЙ ИСТОЧНИК,"1,5л","ключевая, без газа пл/б",NaN,NaN


In [6]:
base_data_dir = Path("/data")

In [7]:
with open(base_data_dir / "train_dataset.json") as f:
    train_dataset = json.load(f)

In [8]:
with open(base_data_dir / "test_dataset.json") as f:
    test_dataset = json.load(f)

In [9]:
print(len(test_dataset), test_dataset[0])

199 {'image_path': 'names/930.jpg', 'text': 'Молоко КРАСНАЯ ЦЕНА\n800мл\nмит.паст.2,5%'}


In [6]:
system_message = "You are an OCR expert specialized in extracting data from supermarket price tags."
user_prompt = "Extract textual information from the image. Output only the text without any reasoning."


In [10]:
def convert_dataset(sample):

   sample_image = Image.open(base_data_dir / sample["image_path"])
   return [
      {"role": "system",
          "content": system_message
      },
      { "role": "user",
        "content": [{
                    "type": "text",
                    "text": user_prompt
                    },
                    {"type": "image", "image": sample_image}
        ]
      },
      { "role": "assistant",
        "content": [{"type": "text",  "text": sample["text"]} ]
      },
      ]

In [20]:
converted_test_dataset = [convert_dataset(sample) for sample in test_dataset]

In [11]:
def process_ocr(image_path, model, temperature = 1.5):
    FastVisionModel.for_inference(model)  # Enable for inference!
    time_start = time.time()
    image = Image.open(base_data_dir / image_path)
    if image.height < 28 or image.width < 28:
      return "!!!ОШИБКА. Ихображение меньше 28точек", 0
      
    plt.imshow(image)
    plt.show()

    messages = [
        {"role": "system",
          "content": system_message
        },
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": user_prompt}
        ]}
    ]
    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

    inputs = tokenizer(
        [image],  # Убедитесь, что это список
        [input_text],
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    input_ids = inputs["input_ids"][0]

    output_ids = model.generate(
        **inputs,
        max_new_tokens=128,
        use_cache=True,
        temperature=temperature,
        min_p=0.1,
    )[0]

    # Удаляем prompt, оставляя только сгенерированное
    generated_only_ids = output_ids[len(input_ids):]


    # Преобразуем выход модели в текст
    recognized_text = tokenizer.decode(generated_only_ids, skip_special_tokens=True)

    time_end = time.time()
    return recognized_text, time_end - time_start


## Qwen2.5 3b

In [9]:
model_path = "/data/models/qwen2_5_3b_outputs"
model, tokenizer = FastVisionModel.from_pretrained(
    model_path,
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)


==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A2. Num GPUs = 1. Max memory: 15.617 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
df_result = pd.DataFrame(columns=["file_path", "original_txt", "recognized_txt", "eot"])  
for i, sample in enumerate(test_dataset):
    original_txt = sample["text"]
    recognized_txt, eot = process_ocr(sample["image_path"], model)
    file_name = sample["image_path"]

    # Добавляем строку в DataFrame
    print("Оригинальный текст:", original_txt)
    print("Распознанный текст:", recognized_txt)
    print("Затраченное время:", eot)
    df_result.loc[i] = [file_name, original_txt, recognized_txt, eot]


In [ ]:
df_result.to_excel(base_data_dir / "qwen2.5_3b_finetuned.xlsx", index=False)